# Creating Web Maps in Python with GeoPandas and Folium

## Introduction


In this post, I demonstrate the use of the Python package Folium to create a web map from a GeoDataFrame. Folium is built on the Leaflet javascript library, which is a great tool for creating interactive web maps. However, I use Python for all of my data wrangling and analytical tasks, so it's really nice to be able to have the web-mapping capabilities from within the same environment. The goal of this post is to demonstrate a workflow between GeoPandas and Folium that makes it really easy to create functional and visually appealing web maps in Python.

In this example, I plot the point locations of crimes in San Francisco, overlaid on a choropleth of census tract crime density. Viewing these two layers together on a web map creates a nice way to get an overall sense of crime distribution, while also being able to view individual crime information. As I demonstrate below, these Python packages provide a nice, clean, and customizable way of doing this.

In [1]:
#Import the necessary Python moduless
import pandas as pd
import geopandas as gpd
import numpy as np
from geopandas.tools import sjoin
import folium
from folium.plugins import MarkerCluster
from folium import IFrame
import shapely
from shapely.geometry import Point
import unicodedata
import pysal as ps

## Data Prep
In this section I create two GeoDataFrames: one of crime points and one of census tract boundaries with crime densities. Both of these will then be plotted on a web map as separate layers.

### Read in Crime Data and Create a GeoDataFrame
First I read in a CSV file of San Francisco Police Incidents for the current year into a Pandas DataFrame. I downloaded the raw data from the San Francisco [Open Data Portal](https://data.sfgov.org/). Because there are so many crime incidents, I select a subset of the data: crimes in the "assault" category that were committed in the last 10 days. As shown below, this leaves me with 329 police incidents. 


In [2]:
#Read in CSV file specifying date field and encoding. Sort by date
all_shootings = pd.read_csv('All_Shooting_Incidents_for_Deputy_Involved_Shootings_2010_to_Present.csv', \
                        encoding = 'utf-8').reset_index(drop=True)

#Clean up the encoding on the Crime Description field 
#all_crime.Descript = all_crime.Descript.apply(lambda x: unicodedata.normalize("NFKD", x))

#Create a field that contains a string representation of the date, for later use
#all_crime['DateStr'] = all_crime.Date.apply(lambda x: x.strftime("%Y-%m-%d"))

#Identify those crimes that are categorized as assaults
animal_shootings = all_shootings['INCIDENT TYPE'] == 'ANIMAL SHOOTING INCIDENT' 

#Identify those crimes that were committed in the most recent 10 days of the dataset
#recent = all_crime.Date.isin(all_crime.Date.unique()[-10:]) 

#Subset the data to get assaults commited within the last 10 days
rex_shot = all_shootings[animal_shootings].drop_duplicates('INCIDENT NUMBER').reset_index(drop = True)
#animal_shootings = animal_shootings[['INCIDENT NUMBER', 'Descript', 'DateStr', 'Time', 'Address','X', 'Y']]
print (f'{len(rex_shot)} Animal Shooting Incidents out of {len(all_shootings)} Deputy Involved Incidents')
rex_shot = rex_shot[['INCIDENT NUMBER', 'INCIDENT TYPE', 'GEO_LOCATION','APPROX_LATITUDE', 'APPROX_LONGITUDE']]
rex_shot.head()

205 Animal Shooting Incidents out of 656 Deputy Involved Incidents


,INCIDENT NUMBER,INCIDENT TYPE,GEO_LOCATION,APPROX_LATITUDE,APPROX_LONGITUDE
0,1000169,ANIMAL SHOOTING INCIDENT,"16100 S. ESSEY AVENUE\nEAST RANCHO DOMINGUEZ, ...",33.887836,-118.204052
1,1000091,ANIMAL SHOOTING INCIDENT,"4900 SUNBURST DRIVE\nPALMDALE, CA 93552\n(34.5...",34.552720,-118.042624
2,1000143,ANIMAL SHOOTING INCIDENT,"9500 BANTA ROAD\nPICO RIVERA, CA 90660\n(34.01...",34.013747,-118.071077
3,1000102,ANIMAL SHOOTING INCIDENT,"3100 MARBELLA LANE\nPALMDALE, CA 93550\n(34.57...",34.570721,-118.073199
4,1001244,ANIMAL SHOOTING INCIDENT,"1700 E. AVENUE Q-14\nPALMDALE, CA 93550\n(34.5...",34.574401,-118.098330


I now want to convert the assault data Pandas DataFrame to a GeoPandas GeoDataFrame (a spatial version of the former). The raw crime data comes with lat/long coordinates, which I use these to create Shapely point geometry objects (these are the values in the "geometry" field for each record in a GeoDataFrame). I specify the spatial reference system as ESPG 4326 which represents the standard WGS84 coordinate system.

In [3]:
#First create a GeoSeries of crime locations by converting coordinates to Shapely geometry objects
#Specify the coordinate system ESPG4326 which represents the standard WGS84 coordinate system
shootings_geo = gpd.GeoSeries(rex_shot.apply(lambda z: Point(z['APPROX_LONGITUDE'], z['APPROX_LATITUDE']), 1),crs={'init': 'epsg:4269'})

#Create a geodataframe from the pandas dataframe and the geoseries of shapely geometry objects
shootings = gpd.GeoDataFrame(rex_shot.drop(['APPROX_LATITUDE', 'APPROX_LONGITUDE'], 1), geometry=shootings_geo)
shootings.head()

,INCIDENT NUMBER,INCIDENT TYPE,GEO_LOCATION,geometry
0,1000169,ANIMAL SHOOTING INCIDENT,"16100 S. ESSEY AVENUE\nEAST RANCHO DOMINGUEZ, ...",POINT (-118.204052 33.887836)
1,1000091,ANIMAL SHOOTING INCIDENT,"4900 SUNBURST DRIVE\nPALMDALE, CA 93552\n(34.5...",POINT (-118.042624 34.55272)
2,1000143,ANIMAL SHOOTING INCIDENT,"9500 BANTA ROAD\nPICO RIVERA, CA 90660\n(34.01...",POINT (-118.071077 34.013747)
3,1000102,ANIMAL SHOOTING INCIDENT,"3100 MARBELLA LANE\nPALMDALE, CA 93550\n(34.57...",POINT (-118.073199 34.570721)
4,1001244,ANIMAL SHOOTING INCIDENT,"1700 E. AVENUE Q-14\nPALMDALE, CA 93550\n(34.5...",POINT (-118.09833 34.574401)


### Calculate Census Tract Crime Density
Next, I read in a Shapefile of San Francisco Census Tracts, which I also downloaded from the SF Open Data Portal. With GeoPandas, I can read a Shapefile directly into Python really easily. Then in one line of code, I spatially join census tracts to assaults (determine the census tract of each assault), and generate counts of assaults per census tract. Note that I use the ```to_crs``` function to convert assaults to the same coordinate system as Census Tracts (EPSG 3310) prior to spatially joining them.

Lastly, I calculate the number of assaults per square mile, which is the metric that I'm interested in plotting.

In [25]:
#Read tracts shapefile into GeoDataFrame
tracts = gpd.read_file('CENSUS_TRACTS_2010.shp')
tracts = tracts[['CT10','geometry']]
#tracts = tracts.set_index('CT10')
#tract.reset_index()
#Generate Counts of Assaults per Census Tract
#Spatially join census tracts to assaults (after projecting) and then group by Tract FIPS while counting the number of crimes
tract_counts = gpd.tools.sjoin(shootings.to_crs(tracts.crs), tracts.reset_index())
tract_counts = tract_counts.groupby('CT10').size()
#Calculate Assault Density, converting square meters to square miles.
tracts['CopShotRexPSqMi'] = (tract_counts/(tracts.geometry.area*3.86102e-7)).fillna(0)
#tracts = tracts.reset_index()

tracts = tracts.loc[tracts['CopShotRexPSqMi'] > 0]
tracts

,CT10,geometry,CopShotRexPSqMi


In [7]:
tracts = gpd.read_file('CENSUS_TRACTS_2010.shp')
tracts = tracts[['CT10','geometry']]
tracts = tracts.set_index('CT10')
tracts.geometry.area
tracts.geometry.area*3.86102e-7

CT10
911001    1554.344599
980003     802.586311
930301    4292.689111
573003       1.862557
297602       2.585737
297601       3.083467
577504       2.140656
576903       1.349627
576302       1.345476
576301       2.279971
576001       7.474381
293306       2.612725
295103       8.554243
980015      22.854327
670602      58.972651
670416      14.866914
670328      14.886201
670326      16.752821
670324      19.686885
651402       7.604295
651401       4.176630
297202       2.393326
297201       1.679114
296902       2.039739
296901       2.565993
296402       4.344135
296401       4.106699
294701       7.281953
294421       2.762428
294302       3.492735
             ...     
404301       4.602440
406102       6.842924
401002       6.123246
400800      10.938967
910001     306.392039
403702       7.115182
403408      13.183732
408626      10.323643
401001       6.025421
401102       5.328330
401101      12.531731
401500      15.672095
401203       7.914349
401201       9.729923
40390

## Using Folium to Plot Data
The general approach I take here is to first create a Folium basemap and then add two layers to it: (1) a choropleth of census tracts, symbolized crime density, and (2) crime point locations. I write a separate function to plot each of these two layers, each of which takes a GeoDataFrame as its input. Folium takes unprojected lat/long coordinates for all of its plotting, so I make sure to convert all of my projected GeoDataFrames to WGS84 within the plotting functions.

###  Choropleth Layer of Tract Crime Density
As its inputs, my choropleth function takes a Folium map object, a GeoDataFrame, the name of the feature ID field, and the name of the field whose values will be symbolized. 

Leaflet uses GeoJSON objects to plot vector geometries (GeoJSON is a data format that is used to represent geographical features along with their non-spatial attributes). GeoPandas has a ```to_json``` method which I use to convert the GeoDataFrame to GeoJSON to be used as one of the inputs to Folium's choropleth function. I also specify the id field, which is used to link the geometry in the GeoJSON with the data in the GeoDataFrame.

The function also takes optional parameters for fill color, fill opacity, line opacity, number of classifiers, and classification scheme. All of these have default values if not specified. Folium / Leaflet uses the [Color Brewer](http://colorbrewer2.org/#type=sequential&scheme=YlOrRd&n=5) sequential color schemes, which can easily be specified to view different combinations. 

Lastly, I allow the user to specify the number of classes and the classification scheme. At this point, Folium has limited map classification options, so I instead use Pysal's choropleth [map classfication module](http://pysal.readthedocs.io/en/latest/library/esda/mapclassify.html) to provide some basic classification options. My function defaults to "Fisher_Jenks", but also has options for "Equal_Interval", and "Quantiles". 

Below, I first create a basemap that is centered in San Francisco, and then I run my function on this basemap specifying the Census Tract ID Field as well as the field I want to symbolize on (Assaults Per Square Mile).

In [ ]:
#Create SF basemap specifying map center, zoom level, and using the default OpenStreetMap tiles
crime_map = folium.Map([34, -118], zoom_start = 9)

def add_choropleth(mapobj, gdf, id_field, value_field, fill_color = 'YlOrRd', fill_opacity = 0.6, 
                    line_opacity = 0.2, num_classes = 1, classifier = 'Fisher_Jenks'):
    #Allow for 3 Pysal map classifiers to display data
    #Generate list of breakpoints using specified classification scheme. List of breakpoint will be input to choropleth function
    if classifier == 'Fisher_Jenks':
        threshold_scale=ps.esda.mapclassify.Fisher_Jenks(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Equal_Interval':
        threshold_scale=ps.esda.mapclassify.Equal_Interval(gdf[value_field], k = num_classes).bins.tolist()
    if classifier == 'Quantiles':
        threshold_scale=ps.esda.mapclassify.Quantiles(gdf[value_field], k = num_classes).bins.tolist()
    
    #Convert the GeoDataFrame to WGS84 coordinate reference system
    gdf_wgs84 = gdf.to_crs({'init': 'epsg:4326'})
    
    #Call Folium choropleth function, specifying the geometry as a the WGS84 dataframe converted to GeoJSON, the data as 
    #the GeoDataFrame, the columns as the user-specified id field and and value field.
    #key_on field refers to the id field within the GeoJSON string
    mapobj.choropleth(
                geo_data = gdf_wgs84.to_json(), 
                data = gdf,
                columns = [id_field, value_field], 
                key_on = 'feature.properties.{}'.format(id_field),
                fill_color = fill_color, 
                fill_opacity = fill_opacity, 
                line_opacity = line_opacity,  
                threshold_scale = threshold_scale)
    return mapobj

#Update basemap with choropleth
crime_map=add_choropleth(crime_map, tracts, 'TRACTCE','DeputyInvolvedAnimalShootingsPSqMi')

### Create Crime Point Cluster Layer
Before displaying my map, I will also add the layer of crime point locations. Rather than display each individual point, I use Leaflet's marker clustering feature, which makes it easier to visualize large numbers of points by grouping together those that are close to each other. Additionally, I use popups to display information about a particular crime when the user clicks on a point. Folium lets you create HTML-rich popups called IFrames. I use this feature only in the most basic form, just to display a few lines of information: crime description, date, time, and address. There are obviously much more creative things that can be done with an IFrame popup (tables, graphs, sub-maps, etc.) but for my purposes this is all I need. 

My function takes as its inputs a Folium map object, a GeoDataFrame, and a list of fields to include in the popup. I run this function on my previously created map object (already updated with a choropleth layer), specifying 4 fields of interest that I want to display. 


In [ ]:
def add_point_clusters(mapobj, gdf, popup_field_list):
    #Create empty lists to contain the point coordinates and the point pop-up information
    coords, popups = [], [] 
    #Loop through each record in the GeoDataFrame
    for i, row in gdf.iterrows():
        #Append lat and long coordinates to "coords" list
        coords.append([row.geometry.y, row.geometry.x])
        #Create a string of HTML code used in the IFrame popup
        #Join together the fields in "popup_field_list" with a linebreak between them
        label = '<br>'.join([row[field] for field in popup_field_list])
        #Append an IFrame that uses the HTML string to the "popups" list 
        popups.append(IFrame(label, width = 300, height = 100))
        
    #Create a Folium feature group for this layer, since we will be displaying multiple layers
    pt_lyr = folium.FeatureGroup(name = 'pt_lyr')
    
    #Add the clustered points of crime locations and popups to this layer
    pt_lyr.add_child(MarkerCluster(locations = coords, popups = popups))
    
    #Add this point layer to the map object
    mapobj.add_child(pt_lyr)
    return mapobj

#Update choropleth with point clusters
crime_map = add_point_clusters(crime_map, assaults, ['Descript','Address','DateStr','Time'])

### Add Layer Control, Display and Save Map
As a finishing touch, I add Layer Control to the map, which allows me to toggle on/off either of my two layers (see widget on the top right). Then I save my finished map as an HTML and display it! 

I hope this was helpful in demonstrating some of the mapping capabilities of Leaflet accessed through the package Folium.  The functions I wrote provide a nice way of displaying two very common types of spatial data on a basemap and can obviously be tweaked to add more custom functionality.

In [ ]:
folium.LayerControl().add_to(crime_map) #Add layer control to toggle on/off
crime_map.save('sf_assaults.html') #save HTML
crime_map #display map